In [26]:
# Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing and Model Selection
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler, PolynomialFeatures
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

# Evaluation Metrics
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, classification_report, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Pipeline
from sklearn.pipeline import Pipeline, make_pipeline

# Saving models
import joblib
import os

In [27]:

def read_files_to_dataframe(directory, years):
    data_frames = []

    for year in years:
        file_path = os.path.join(directory, f"{year}.xlsx")
        try:
            # Read the Excel file into a DataFrame
            df = pd.read_excel(file_path)
            df['Year'] = year  # Add a column for the year
            data_frames.append(df)
        except FileNotFoundError:
            print(f"File {file_path} not found. Skipping.")
        except Exception as e:
            print(f"Error reading {file_path}: {e}")

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df


data_dir = "../Raw_historical_data"
file_years = range(2023, 2025)  

historical_data_df = read_files_to_dataframe(data_dir, file_years)
historical_data_df = historical_data_df[['Date', 'Surface', 'Winner', 'Loser', 'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3','W4','L4','W5','L5', 'AvgW', 'AvgL']]

print(historical_data_df.head())



        Date Surface         Winner               Loser    WPts    LPts   W1  \
0 2023-01-01    Hard       Giron M.          Gasquet R.   776.0   740.0  7.0   
1 2023-01-01    Hard    Mcdonald M.          Galan D.E.   775.0   741.0  6.0   
2 2023-01-02    Hard  Kecmanovic M.        O Connell C.  1420.0   652.0  6.0   
3 2023-01-02    Hard    Nishioka Y.             Rune H.  1134.0  2888.0  2.0   
4 2023-01-02    Hard     Popyrin A.  Auger-Aliassime F.   469.0  4195.0  6.0   

    L1   W2   L2   W3   L3  W4  L4  W5  L5  AvgW  AvgL  
0  6.0  6.0  7.0  7.0  5.0 NaN NaN NaN NaN  1.89  1.89  
1  3.0  NaN  NaN  NaN  NaN NaN NaN NaN NaN  1.36  3.12  
2  4.0  6.0  4.0  NaN  NaN NaN NaN NaN NaN  1.58  2.36  
3  6.0  6.0  4.0  6.0  4.0 NaN NaN NaN NaN  3.56  1.29  
4  4.0  7.0  6.0  NaN  NaN NaN NaN NaN NaN  6.04  1.13  


In [32]:
# Create a copy to flip every second row
flipped_df = historical_data_df.copy()

# Flip Winner/Loser columns in every second row
flipped_df.loc[1::2, ['Winner', 'Loser']] = historical_data_df.loc[1::2, ['Loser', 'Winner']].values

# Flip points, ranks, and set scores
flipped_df.loc[1::2, ['WPts', 'LPts']] = historical_data_df.loc[1::2, ['LPts', 'WPts']].values
flipped_df.loc[1::2, ['W1', 'L1']] = historical_data_df.loc[1::2, ['L1', 'W1']].values
flipped_df.loc[1::2, ['W2', 'L2']] = historical_data_df.loc[1::2, ['L2', 'W2']].values
flipped_df.loc[1::2, ['W3', 'L3']] = historical_data_df.loc[1::2, ['L3', 'W3']].values
flipped_df.loc[1::2, ['W4', 'L4']] = historical_data_df.loc[1::2, ['L4', 'W4']].values
flipped_df.loc[1::2, ['W5', 'L5']] = historical_data_df.loc[1::2, ['L5', 'W5']].values
flipped_df.loc[1::2, ['AvgW', 'AvgL']] = historical_data_df.loc[1::2, ['AvgL', 'AvgW']].values

# Create a new Winner column (1 if P1 wins, 0 otherwise)
flipped_df['WinnerLabel'] = (flipped_df['Winner'] == historical_data_df['Winner']).astype(int)

# Rename columns to P1S1, P2S1, etc.
flipped_df = flipped_df.rename(columns={
    'W1': 'P1S1', 'L1': 'P2S1',
    'W2': 'P1S2', 'L2': 'P2S2',
    'W3': 'P1S3', 'L3': 'P2S3',
    'W4': 'P1S4', 'L4': 'P2S4',
    'W5': 'P1S5', 'L5': 'P2S5'
})

flipped_df[['P1S1', 'P2S1', 'P1S2', 'P2S2', 'P1S3', 'P2S3', 'P1S4', 'P2S4', 'P1S5', 'P2S5']] = flipped_df[['P1S1', 'P2S1', 'P1S2', 'P2S2', 'P1S3', 'P2S3', 'P1S4', 'P2S4', 'P1S5', 'P2S5']].fillna(0)


# Rename Winner and Loser to P1 and P2
flipped_df = flipped_df.rename(columns={'Winner': 'P1', 'Loser': 'P2', 'WPts':'P1Pts', 'LPts':'P2Pts', 'AvgW':'AvgP1' , 'AvgL':'AvgP2' })


# Display the first few rows
print(flipped_df.head())

        Date Surface             P1                  P2   P1Pts   P2Pts  P1S1  \
0 2023-01-01    Hard       Giron M.          Gasquet R.   776.0   740.0   7.0   
1 2023-01-01    Hard     Galan D.E.         Mcdonald M.   741.0   775.0   3.0   
2 2023-01-02    Hard  Kecmanovic M.        O Connell C.  1420.0   652.0   6.0   
3 2023-01-02    Hard        Rune H.         Nishioka Y.  2888.0  1134.0   6.0   
4 2023-01-02    Hard     Popyrin A.  Auger-Aliassime F.   469.0  4195.0   6.0   

   P2S1  P1S2  P2S2  P1S3  P2S3  P1S4  P2S4  P1S5  P2S5  AvgP1  AvgP2  \
0   6.0   6.0   7.0   7.0   5.0   0.0   0.0   0.0   0.0   1.89   1.89   
1   6.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   3.12   1.36   
2   4.0   6.0   4.0   0.0   0.0   0.0   0.0   0.0   0.0   1.58   2.36   
3   2.0   4.0   6.0   4.0   6.0   0.0   0.0   0.0   0.0   1.29   3.56   
4   4.0   7.0   6.0   0.0   0.0   0.0   0.0   0.0   0.0   6.04   1.13   

   WinnerLabel  
0            1  
1            0  
2            1  
3     

In [30]:
df = flipped_df

def preprocess_data(first_row_index):

    # Extract unique player names from the dataset
    unique_players = pd.concat([df['P1'], df['P2']]).unique()

    # Initialize dictionaries to store running totals and counts using player names as keys
    total_points_diff = {player: 0 for player in unique_players}
    sets_played = {player: 0 for player in unique_players}
    matches_played = {player: 0 for player in unique_players}
    win_record = {player: [] for player in unique_players}
    h2h_records = {(p1, p2): {'outcomes': [], 'matches': 0} 
                   for p1 in unique_players for p2 in unique_players if p1 != p2}
    surface_stats = {player: {'Hard': {'wins': 0, 'games': 0}, 
                              'Clay': {'wins': 0, 'games': 0}, 
                              'Grass': {'wins': 0, 'games': 0}} 
                     for player in unique_players}

    # Process the initial rows to populate dictionaries
    initial_df = df.iloc[:first_row_index]
    
    #region DICTIONARIES FILLING WITH INITIAL DATA
    for _, row in initial_df.iterrows():

        # Identifying players of the row :
        p1 = row['P1']
        p2 = row['P2']

        # Identifying Player Points :
        ATP_Points_p1 = row['P1Pts']
        ATP_Points_p2 = row['P2Pts']

        # Identifying Surface type :
        surface = row['Surface']

        # Row points score calculation
        points_diff_p1 = (
            (row['P1S1'] - row['P2S1']) +
            (row['P1S2'] - row['P2S2']) +
            (row['P1S3'] - row['P2S3']) +
            (row['P1S4'] - row['P2S4']) +
            (row['P1S5'] - row['P2S5'])
        )

        # Dictionnary points diff counting updating : 
        total_points_diff[p1] += points_diff_p1
        total_points_diff[p2] -= points_diff_p1  

        # Row Sets Number Calculation
        sets_played_p1 = (
            (row['P1S1'] > 0) + 
            (row['P1S2'] > 0) + 
            (row['P1S3'] > 0) + 
            (row['P1S4'] > 0) + 
            (row['P1S5'] > 0)
        )

        sets_played_p2 = (
            (row['P2S1'] > 0) + 
            (row['P2S2'] > 0) + 
            (row['P2S3'] > 0) + 
            (row['P2S4'] > 0) + 
            (row['P2S5'] > 0)
        )

        # Dictionary sets number updating, max in case of 6-0
        sets_played[p1] += max(sets_played_p1, sets_played_p2)
        sets_played[p2] += max(sets_played_p1, sets_played_p2)

        # Dictionary sets number updating.
        matches_played[p1] += 1
        matches_played[p2] += 1

        # Row winner identification:
        winner = row['Winner']

        # Dictionary updating with win or loss for both row players : 
        if np.abs(ATP_Points_p1 - ATP_Points_p2) <= 500:
            win_record[p1].append(1 if winner == 1 else 0)
            win_record[p2].append(1 if winner == 2 else 0)

        # Dictionary updating incrementing matches number for p1 p2 and adding outcomes of game:
        h2h_records[(p1, p2)]['matches'] += 1
        h2h_records[(p2, p1)]['matches'] += 1
        if winner == 1:
            h2h_records[(p1, p2)]['outcomes'].append(1)
            h2h_records[(p2, p1)]['outcomes'].append(0)
        elif winner == 2:
            h2h_records[(p1, p2)]['outcomes'].append(0)
            h2h_records[(p2, p1)]['outcomes'].append(1)

        # Dict updating for Surface Stats only if player Ranks close:
        if np.abs(rank_p1 - rank_p2) <= 50:
            surface_stats[p1][surface]['games'] += 1
            surface_stats[p2][surface]['games'] += 1
            if winner == 1:
                surface_stats[p1][surface]['wins'] += 1
            elif winner == 2:
                surface_stats[p2][surface]['wins'] += 1
    #endregion

    # Initialize a new DataFrame to store the transformed rows
    new_df = pd.DataFrame(columns=[
        'P1',
        'P2',
        'ATP_P1_Pts', 
        'ATP_P2_Pts', 
        'DIFF_ATP_Pts', #### 
        'AVG_P1', 
        'AVG_P2', 
        'DIFF_AVG', ####
        'Num_Data_Points_AVG_P1_P2', 
        'WR_P1',
        'WR_P2',
        'DIFF_WR', ####
        'Num_Data_Points_WR_P1_P2',
        'H2H_P1P2', ####
        'Num_Data_Points_H2H',
        'Surface_WR_P1',
        'Surface_WR_P2',
        'DIFF_Surface_WR_P1_P2', ####
        'Surface',
        'Num_Data_Points_Surface_P1_P2',
        'Winner'
        'B365P1'
        'B365P2'
    ])

    # Loop through each remaining row
    for i in range(first_row_index, len(df)):

        initial_df = df.iloc[:i+1]

        #region USING CURRENT VERSION OF DICTIONARIES TO CREATE PROCESSED CURRENT ROW:

        # Selecting current row and identifying p1 and p2
        game = initial_df.iloc[-1]
        p1 = game['P1']
        p2 = game['P2']

        # Lookup skills for p1 and p2
        rank_p1 = game['RANK_P1']
        rank_p2 = game['RANK_P2']

        # Using processed dictionaries to calculate avg points diff of each player : 
        avg_p1 = total_points_diff[p1] / sets_played[p1] if sets_played[p1] != 0 else 0
        avg_p2 = total_points_diff[p2] / sets_played[p2] if sets_played[p2] != 0 else 0

       
        # Summing the last 10 matches outcomes 1 or 0 :
        wr_p1 = sum(win_record[p1][-10:]) / len(win_record[p1][-10:]) if len(win_record[p1][-10:]) > 0 else 0
        wr_p2 = sum(win_record[p2][-10:]) / len(win_record[p2][-10:]) if len(win_record[p2][-10:]) > 0 else 0


        # Rounding the win-Ratio : 
        wr_p1 = round(wr_p1, 2)
        wr_p2 = round(wr_p2, 2) 

        # Get the data points number for calculating wr : 
        num_points_wr_p1 = len(win_record[p1][-10:])
        num_points_wr_p2 = len(win_record[p2][-10:])

        # Ensure there are at least 10 games or use the available number of matches for H2H
        h2h_matches = min(10, h2h_records[(p1, p2)]['matches'])
        h2h_outcomes = h2h_records[(p1, p2)]['outcomes'][-h2h_matches:]
        h2h_wins = sum(h2h_outcomes)

        # Calculate the H2H ratio
        h2h_p1_p2 = h2h_wins / h2h_matches if h2h_matches > 0 else 0

        # Calculate surface-specific win ratios for the current match's surface
        surface = game['Surface']
        surface_wr_p1 = surface_stats[p1][surface]['wins'] / surface_stats[p1][surface]['games'] if surface_stats[p1][surface]['games'] > 0 else 0
        surface_wr_p2 = surface_stats[p2][surface]['wins'] / surface_stats[p2][surface]['games'] if surface_stats[p2][surface]['games'] > 0 else 0

        num_points_surface_p1 = surface_stats[p1][surface]['games']
        num_points_surface_p2 = surface_stats[p2][surface]['games']

        # Create the transformed row for the current game
        transformed_row = {
            'P1': p1,
            'P2': p2,
            'RANK_P1': rank_p1,
            'RANK_P2' : rank_p2,
            'DIFF_RANK' : rank_p2 - rank_p1,
            'AVG_P1': round(avg_p1,2),
            'AVG_P2': round(avg_p2,2),
            'DIFF_AVG': round(avg_p1-avg_p2,2),
            'Num_Points_AVG_P1_P2': (matches_played[p1],  matches_played[p2]),
            'WR_P1': round(wr_p1,2),
            'WR_P2': round(wr_p2,2),
            'DIFF_WR': round(wr_p1-wr_p2,2),
            'Num_Points_WR_P1_P2': (num_points_wr_p1, num_points_wr_p2),
            'H2H_P1P2': round(h2h_p1_p2,2),
            'Num_Points_H2H': h2h_matches,
            'Surface_WR_P1': round(surface_wr_p1, 2),
            'Surface_WR_P2': round(surface_wr_p2,2),
            'DIFF_Surface_WR_P1_P2': round(surface_wr_p1 - surface_wr_p2,2),
            'Surface': surface,
            'Num_Points_Surface_P1_P2': (num_points_surface_p1, num_points_surface_p2),
            'Winner': game['Winner'],
            'B365P1' : game['B365P1'],
            'B365P2' : game['B365P2']
        }
        
        #endregion

        new_df = pd.concat([new_df, pd.DataFrame([transformed_row])], ignore_index=True)

        #region USING CURRENT ROW DATA TO UPDATE DICTIONNARIES AND BE USED IN NEXT ROW : 

        # Row points score calculation

        points_diff_p1 = (
            (game['P1S1'] - game['P2S1']) +
            (game['P1S2'] - game['P2S2']) +
            (game['P1S3'] - game['P2S3']) +
            (game['P1S4'] - game['P2S4']) +
            (game['P1S5'] - game['P2S5'])
        )

        # Dictionary points diff counting updating : 
        total_points_diff[p1] += points_diff_p1
        total_points_diff[p2] -= points_diff_p1  

        # Row Sets Number Calculation
        sets_played_p1 = (
            (game['P1S1'] > 0) + 
            (game['P1S2'] > 0) + 
            (game['P1S3'] > 0) + 
            (game['P1S4'] > 0) + 
            (game['P1S5'] > 0)
        )

        sets_played_p2 = (
            (game['P2S1'] > 0) + 
            (game['P2S2'] > 0) + 
            (game['P2S3'] > 0) + 
            (game['P2S4'] > 0) + 
            (game['P2S5'] > 0)
        )

        # Dictionary sets number updating, max in case of 6-0
        sets_played[p1] += max(sets_played_p1, sets_played_p2)
        sets_played[p2] += max(sets_played_p1, sets_played_p2)

        # Dictionary sets number updating.
        matches_played[p1] += 1
        matches_played[p2] += 1

        # Row winner identification :
        winner = game['Winner']

        # Dictionary updating with win or loss for both row players : 
        if np.abs(rank_p1 - rank_p2) <= 50:
            win_record[p1].append(1 if winner == 1 else 0)
            win_record[p2].append(1 if winner == 2 else 0)

        # Dictionary updating incrementing matches number for p1 p2 and adding outcomes of game :
        h2h_records[(p1, p2)]['matches'] += 1
        h2h_records[(p2, p1)]['matches'] += 1
        if winner == 1:
            h2h_records[(p1, p2)]['outcomes'].append(1)
            h2h_records[(p2, p1)]['outcomes'].append(0)
        elif winner == 2:
            h2h_records[(p1, p2)]['outcomes'].append(0)
            h2h_records[(p2, p1)]['outcomes'].append(1)


        # Dict updating for Surface Stats only if player Ranks close:
        if np.abs(rank_p1 - rank_p2) <= 50:
            surface_stats[p1][surface]['games'] += 1
            surface_stats[p2][surface]['games'] += 1
            if winner == 1:
                surface_stats[p1][surface]['wins'] += 1
            elif winner == 2:
                surface_stats[p2][surface]['wins'] += 1

        #endregion

    return new_df

# Example usage
new_df = preprocess_data(first_row_index = 3759)
new_df.to_csv('transformed_data.csv', index=False)



KeyError: 'RANK_P1'